In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [3]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

In [4]:
from mcr_py import run_mlc_with_bags, GraphCache
from copy import deepcopy
from typing import Any, Tuple

import pandas as pd
import geopandas as gpd

import mcr_py
from mcr_py import GraphCache
import pyrosm
from package import storage, strtime
from package.logger import Timed
from package.mcr.label import McRAPTORLabel, merge_intermediate_bags
from package.mcr.path import PathManager, PathType
from package.osm import osm, graph
from package.raptor.mcraptor_single import McRaptorSingle
from package.raptor.bag import Bag
from package.mcr.bag import (
    convert_mc_raptor_bags_to_intermediate_bags,
    convert_mlc_bags_to_intermediate_bags,
)

import matplotlib.pyplot as plt
from package.minute_city import minute_city
from package.mcr.data import MCRGeoData
from package.mcr.mcr import MCR

import os
import folium
from package.mcr.path import PathType
from package.geometa import GeoMeta

In [5]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	other_stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

geo_meta = GeoMeta.load("../data/geometa.pkl")

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_boundary(osm_reader, geo_meta)

[09:02:49] INFO     Reading stops                                                                   ]8;id=635824;file:///tmp/ipykernel_6564/1014092602.py\1014092602.py]8;;\:]8;id=238244;file:///tmp/ipykernel_6564/1014092602.py#5\5]8;;\

           INFO     Reading stops done (0.24 seconds)                                               ]8;id=284840;file:///tmp/ipykernel_6564/1014092602.py\1014092602.py]8;;\:]8;id=163995;file:///tmp/ipykernel_6564/1014092602.py#5\5]8;;\

           INFO     Using existing OSM data                                                        ]8;id=122428;file:///tmp/ipykernel_6564/1014092602.py\1014092602.py]8;;\:]8;id=671627;file:///tmp/ipykernel_6564/1014092602.py#12\12]8;;\

           INFO     Getting OSM graph                                                              ]8;id=23300;file:///tmp/ipykernel_6564/1014092602.py\1014092602.py]8;;\:]8;id=199228;file:///tmp/ipykernel_6564/1014092602.py#18\18]8;;\

           INFO     Loading OSM network from cache                                                 ]8;id=963854;file:///tmp/ipykernel_6564/1014092602.py\1014092602.py]8;;\:]8;id=242236;file:///tmp/ipykernel_6564/1014092602.py#18\18]8;;\

[09:02:51] INFO     Getting OSM graph done (2.55 seconds)                                          ]8;id=850261;file:///tmp/ipykernel_6564/1014092602.py\1014092602.py]8;;\:]8;id=361428;file:///tmp/ipykernel_6564/1014092602.py#18\18]8;;\

In [6]:
import folium


m = folium.Map(location=geo_meta.get_center_lat_lon(), zoom_start=12)
for node in nodes.sample(1000).itertuples():
	folium.CircleMarker(
		location=[node.lat, node.lon],
		color="red",
		radius=1,
		fill=True,
		fill_color="red",
	).add_to(m)


m 

In [7]:
pois = minute_city.fetch_pois_for_area(nodes.unary_union.convex_hull, nodes)

[09:03:40] WARNING  Found 731 entries with distance to nearest OSM node larger than 1000 meters          ]8;id=875190;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=413600;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#194\194]8;;\

           WARNING  lat: 50.9975452, lon: 6.8037577, nearest_osm_node_id: 431042787, distance:           ]8;id=805664;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=649772;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    3695.5226530253135                                                                             

           WARNING  lat: 50.9573667, lon: 7.1150245, nearest_osm_node_id: 1829904993, distance:          ]8;id=667248;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=365019;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    1466.5118258882899                                                                             

           WARNING  lat: 50.9981784, lon: 6.8018351, nearest_osm_node_id: 431042787, distance:           ]8;id=752098;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=873397;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    3902.5345694921566                                                                             

           WARNING  lat: 50.9975499, lon: 6.8034524, nearest_osm_node_id: 431042787, distance:           ]8;id=381062;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=171484;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    3729.2434380196423                                                                             

...

           WARNING  lat: 51.0319755, lon: 7.0118402, nearest_osm_node_id: 2536738887, distance:          ]8;id=76223;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=898471;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    1735.6061758585645                                                                             

           WARNING  lat: 50.9563173, lon: 7.1163925, nearest_osm_node_id: 1829904993, distance:          ]8;id=843590;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=546929;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    1316.3433367304938                                                                             

           WARNING  lat: 50.8325007, lon: 7.0308556, nearest_osm_node_id: 2652538088, distance:          ]8;id=215060;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=206282;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    1400.6263762575122                                                                             

           WARNING  lat: 50.9593915, lon: 7.1081768, nearest_osm_node_id: 25721536, distance:            ]8;id=44744;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=86225;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    1226.363136546344                                                                              

           WARNING  lat: 50.8331714, lon: 7.0312803, nearest_osm_node_id: 2652538088, distance:          ]8;id=580855;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=103901;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#198\198]8;;\
                    1383.7145195631713                                                                             

In [8]:
poi

NameError: name 'pis' is not defined

In [9]:
stops_path = "../data/cleaned/stops.csv"
city_id = "Koeln"
osm_path = ""
structs = "../data/structs.pkl"

mcr_geo_data = MCRGeoData(
    stops_path, structs, city_id, bicycle_location_path="../data/bicycle_locations.csv"
)


[17:14:10] INFO     Reading stops and geo meta                                                           ]8;id=273284;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=272182;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#40\40]8;;\

           INFO     Reading stops and geo meta failed (0.10 seconds)                                     ]8;id=873541;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=235802;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#40\40]8;;\

FileNotFoundError: [Errno 2] No such file or directory: 'Koeln'

In [ ]:
mcr_geo_data.add_pois_to_mm_graph(pois)

In [ ]:
mcr = MCR(mcr_geo_data, disable_paths=False, enable_limit=False)
mcr.run(1638242852, "08:00:00", 2, output_path="../data/bags.pkl")

[18:00:11] INFO     Running Dijkstra step                                                              ]8;id=53340;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=442989;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[18:00:13] INFO     Running Dijkstra step done (1.87 seconds)                                          ]8;id=333415;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=759255;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Extracting dijkstra step bags                                                      ]8;id=113013;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=562207;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[18:00:18] INFO     Extracting dijkstra step bags done (4.77 seconds)                                  ]8;id=797195;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=967836;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Running iteration 1                                                                   ]8;id=797247;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/mcr.py\mcr.py]8;;\:]8;id=917929;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/mcr.py#97\97]8;;\

...

           INFO     Extracting walking step bags                                                       ]8;id=344437;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=280063;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[18:02:00] INFO     Extracting walking step bags done (7.12 seconds)                                   ]8;id=757904;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=653572;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Saving bags                                                                        ]8;id=308500;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=778983;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[18:02:11] INFO     Saving bags done (10.62 seconds)                                                   ]8;id=683805;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=517610;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

: 